#Classical Approach

In [1]:
!ls

sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My\ Drive/

/content/drive/My Drive


In [0]:
!ls

In [4]:
cd NLP_Assignment_2

/content/drive/My Drive/NLP_Assignment_2


In [0]:
!wget https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt

--2019-10-05 18:23:54--  https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 924745 (903K) [text/plain]
Saving to: ‘speeches.txt’

speeches.txt        100%[===================>] 903.07K  --.-KB/s    in 0.05s   

2019-10-05 18:23:54 (18.2 MB/s) - ‘speeches.txt’ saved [924745/924745]



In [9]:
!pip install nltk --upgrade

     |████████████████████████████████| 1.5MB 2.7MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449910 sha256=5d4544d6ae1150782cbfb358015788cb23b0a2fc75d513dd00b14cb0b96169d6
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
file = open('speeches.txt',encoding="utf-8")
text = file.read()


In [11]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
print(nltk.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
3.2.5


In [0]:
vocablist = []

In [13]:
sentences = sent_tokenize(text)
print(len(sentences))


16493


In [0]:
for i in range(len(sentences)) : 
  #print(i)
  k = re.sub(r'[^A-Za-z\s\']',"",sentences[i])
  k = k.lower()
  sentences[i] = '<s> '+k +' </s>'
  
  
  #print(sentences[i])
  

In [0]:
# for i in sentences:
#   print(i)

In [0]:
train = sentences[:13194]
test = sentences[13194:]

In [17]:
#vocabulary = set(vocablist)

vocabulary = set((' '.join(sentences)).split())

print(len(vocabulary))

6071


#computing unigrams, bigrams etc

In [18]:
from nltk import ngrams
unigramlist = []
bigramlist = []
trigramlist = []
fourgramlist = []
for sentence in train:
  unigramlist += sentence.split()
  bigramlist += list(ngrams(sentence.split(),2))
  trigramlist += list(ngrams(sentence.split(),3))
  fourgramlist += list(ngrams(sentence.split(),4))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: generator 'ngrams' raised StopIteration
  # Remove the CWD from sys.path while we load stuff.


In [0]:
# for i in trifreq.keys():
#   print(i,trifreq[i])

In [0]:
unifreq = nltk.FreqDist(unigramlist)
bifreq =  nltk.FreqDist(bigramlist)
trifreq = nltk.FreqDist(trigramlist)
fourfreq = nltk.FreqDist(fourgramlist)

In [105]:
def unigram_mle(word): #unigram MLE
  if word in unigramlist:
    return((unifreq[word]+1)/(len(unigramlist)+len(vocabulary)))
  else:
    return(1/(len(unigramlist)+len(vocabulary)))
unigram_mle('was')

0.004750320024752625

In [106]:
def bigram_mle(A,B): #Bigram MLE 
  if (B,A) in bigramlist :
    return((1+bifreq[(B,A)])/(unifreq[B] + len(set(unifreq.keys()))))
  else:
    return(1/(unifreq[B]+ len(set(unifreq.keys()))))
bigram_mle('so','you')

0.0007624857033930614

In [107]:
def trigram_mle(A,B,C): #Trigram MLE 
  if (B,C,A) in trigramlist :
    #return((1+trifreq[(B,C,A)])/(bifreq[(B,C)]+ len(set(bifreq.keys()))))
    return((1+trifreq[(B,C,A)])/(bifreq[(B,C)]+ len(set(unifreq.keys()))))    
  else:
    return((1)/(bifreq[(B,C)]+ len(set(unifreq.keys()))))
    
print(bifreq[('thats','not')])
print(trifreq[('thats','not','it')])
#trigram_perplexity(test[3])
trigram_mle('true','thats','not')

  


33
0


0.0001841959845275373

In [108]:
def fourgram_mle(A,B,C,D): #fourgram MLE 
  if (B,C,D,A) in fourgramlist :
    return((1+fourfreq[(B,C,D,A)])/(trifreq[(B,C,D)]+len(set(unifreq.keys()))))
  else:
    return((1)/(trifreq[(B,C,D)]+ len(set(trifreq.keys()))))
fourgram_mle('so','thank','you','yaar')

1.2268132299538719e-05

#perplexities

In [114]:
import math
def unigram_perplexity(sen):
  words = sen.split()
  p = 1
  for i in range(len(words)):
    k = unigram_mle(words[i])
    p = p*(1/k)**(1/len(words))
    #print(p)
  return(p)


#print(bigram_perplexity(test[19]))

def total_perplexity_unigram(test):
  k = 1
  mo = 0
  for i in test: 
    temp = unigram_perplexity(i)
    #print(temp)
    
    k = k* (temp**(1/len(test)))
  return(k)

perplexity_unigram = total_perplexity_unigram(test)
print("perplexity for unigram model",perplexity_unigram)

perplexity for unigram model 245.1713515224132


In [111]:
import math
def bigram_perplexity(sen):
  words = sen.split()
  p = 1
  for i in range(1,len(words)):
    k = bigram_mle(words[i],words[i-1])
    p = p*(1/k)**(1/len(words))
    #print(p)
  return(p)


#print(bigram_perplexity(test[19]))

def total_perplexity_bigram(test):
  k = 1
  mo = 0
  for i in test: 
    temp = bigram_perplexity(i)
    #print(temp)
    
    k = k* (temp**(1/len(test)))
  return(k)

perplexity_bigram = total_perplexity_bigram(test)
print("perplexity for bigram model",perplexity_bigram)

perplexity for unigram model 243.27725505222895


In [112]:
import math
def trigram_perplexity(sen):
  p = 1
  words = sen.split()
  for i in range(2,len(words)):
    #p = p*trigram_mle(sen[i],sen[i-1])
    k = trigram_mle(words[i],words[i-2],words[i-1])
    #print(words[i])
    p = p*(1/k)**(1/len(words))
    #print(p)
  return(p)

#print(train[3])


#trigram_perplexity(train[3])



def total_perplexity_trigram(test):
  k = 1
  for i in test: 
    temp = trigram_perplexity(i)
    #print(temp)
    k = k* (temp**(1/len(test)))
  return(k)

perplexity_trigram = total_perplexity_trigram(test)
print("perplexity for trigram model",perplexity_trigram)

perplexity for trigram model 326.5206426759734


In [113]:
import math
def fourgram_perplexity(sen):
  p = 1
  words = sen.split()
  for i in range(3,len(words)):
    #p = p*trigram_mle(sen[i],sen[i-1])
    k = fourgram_mle(words[i],words[i-3],words[i-2],words[i-1])
    #print(words[i])
    p = p*(1/k)**(1/len(words))
    #print(p)
  return(p)

#print(train[3])


#trigram_perplexity(train[3])



def total_perplexity_fourgram(test):
  k = 1
  for i in test: 
    temp = fourgram_perplexity(i)
    #print(temp)
    k = k* (temp**(1/len(test)))
  return(k)

perplexity_fourgram = total_perplexity_fourgram(test)
print("perplexity for fourgram model",perplexity_fourgram)

perplexity for fourgram model 751.5439496710676


#Text generation

In [0]:
import random
def generate(n):
  
  if n == 2:
    sentence = []
    try:
      k = random.randint(1,len(unigramlist))
      sentence.append(unigramlist[k])
      k = 0
      while k<8:
        #print(k)
        lst1 = []
        lst2 = []

        for bi in bigramlist:
          #print(bi[0])
          if bi[0]==sentence[-1]:
            lst1.append(bi[1])
            lst2.append(bigram_mle(bi[1],bi[0]))
        #print(lst1)
        max1 = lst2.index(max(lst2))
        sentence.append(lst1[max1]) 
        k = k+1
      return(sentence)
    except:
      return(sentence)

  if n == 3:
    sentence = []
    try:
      k = random.randint(1,len(bigramlist))
      sentence.append(bigramlist[k][0])
      sentence.append(bigramlist[k][1])
      k = 0
      #print(sentence)
      while k<10:
        #print(k)
        lst1 = []
        lst2 = []
        for tri in trigramlist:
          #print(bi[0])
          if tri[0]==sentence[-2] and tri[1]==sentence[-1] :
            #print('yo')
            #print(tri[2])
            lst1.append(tri[2])
            #print("appended")
            
            lst2.append(trigram_mle(tri[2],tri[0],tri[1]))
            #print("appended2")
        
        max1 = lst2.index(max(lst2))
        sentence.append(lst1[max1]) 
        k = k+1
      return(sentence)
    except:
      return(sentence)
    
  if n == 4:
    sentence = []
    try:
      k = random.randint(1,len(trigramlist))
      sentence.append(trigramlist[k][0])
      sentence.append(trigramlist[k][1])
      sentence.append(trigramlist[k][2])
      k = 0
      #print(sentence)
      while k<10:
        #print(k)
        lst1 = []
        lst2 = []
        for four in fourgramlist:
          
          if four[0]==sentence[-3] and four[1]==sentence[-2] and four[2]==sentence[-1] :
            lst1.append(four[3])
            #print('yo')
            lst2.append(fourgram_mle(four[3],four[0],four[1],four[2]))
           
        
        max1 = lst2.index(max(lst2))
        sentence.append(lst1[max1]) 
        k = k+1
      return(sentence)
    except:
      return(sentence)

    

def generator(no,ngram):
  if ngram==2:
    for i in range(no):
      print(generate(2))
    return
  if ngram ==3:
    for i in range(no):
      a = generate(3)
      print(a)
  if ngram ==4:
    for i in range(no):
      print (generate(4))
      #print(a)
    return

    

In [174]:
print(generator(5,2)) #generating sentences from bigram model


['</s>']
['going', 'to', 'be', 'a', 'lot', 'of', 'the', 'way', '</s>']
['</s>']
['about', 'it', '</s>']
['going', 'to', 'be', 'a', 'lot', 'of', 'the', 'way', '</s>']
None


In [159]:
print(generator(5,3)) # generating sentences from trigram model

['to', 'me', '</s>']
['with', 'iran', '</s>']
['i', 'went', 'to', 'the', 'united', 'states', '</s>']
['they', 'want', 'to', 'do', 'it', '</s>']
["can't", 'let', 'iran', 'get', 'a', 'lot', 'of', 'people', '</s>']
None


In [168]:
print(generator(5,4)) # generating sentences from quadgram model

['want', 'your', 'support', 'but', 'i', 'dont', 'want', 'to', 'tell', 'you', '</s>']
['<s>', 'is', 'that', 'okay', '</s>']
['the', 'next', 'four', 'years', '</s>']
['came', 'in', 'three', 'minutes', 'ago', 'in', 'new', 'hampshire', '</s>']
['he', 'didnt', 'win', 'anything', '</s>']
None
